In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: (x > 0.5).float()),
])
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

class CNNAutoencoder(nn.Module):
    """
    Un autoencodeur convolutionnel simple:
         - Encoder: 2 blocs Conv -> MaxPool
         - Decoder: 2 blocs ConvTranspose
    """
    def __init__(self):
        super(CNNAutoencoder, self).__init__()
        
        # --- Encoder ---
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Sortie: (B,32,14,14)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)  # Sortie: (B,64,7,7)
        )
        
        # --- Decoder ---
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2),  # (B,32,14,14)
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=2, stride=2)    # (B,1,28,28)
            # Pas de sigmoid ici; on utilisera BCEWithLogitsLoss
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Création d'un modèle vide
loaded_model = CNNAutoencoder().to(device)

# Chargement des poids sauvegardés
loaded_model.load_state_dict(torch.load("mnist_autoencoder.pth", map_location=device))

# Passage du modèle en mode évaluation
loaded_model.eval()
def random_drop_784(batch_images_flat, drop_probability):
    """
    Supprime aléatoirement un pourcentage de bits (0 ou 1) dans chaque image et remplacer par -1.
    """
    mask = (torch.rand_like(batch_images_flat) > drop_probability)
    dropped_flat = batch_images_flat.clone()
    dropped_flat[mask == 0] = -1  # on remplace les bits manquants et non reçus par -1.
    return dropped_flat
    
def test_reconstruction(model, image_tensor, drop_probability=0.3):
    """
    Reçoit une seule image (1,1,28,28),
    remplace certains pixels par -1 (drop),
    la passe à travers le modèle et retourne l'image reconstruite.
    """
    model.eval()  # Mode évaluation
    
    # 1) Aplatissement
    flat = image_tensor.view(1, -1)  # (1,784)
    
    # 2) Application du drop
    dropped_flat = random_drop_784(flat, drop_probability)
    
    # 3) Reformatage en (1,1,28,28)
    dropped_reshaped = dropped_flat.view(1, 1, 28, 28)
    
    # 4) Passage à travers le modèle (avec torch.no_grad pour éviter le calcul des gradients)
    with torch.no_grad():
        logits = model(dropped_reshaped)
        reconstructed = torch.sigmoid(logits)  # Car BCEWithLogitsLoss a été utilisé
    
    return dropped_reshaped, reconstructed

def save_reconstruction(original, dropped, reconstructed, prefix="output"):
    """
    Enregistre chaque les images.
    """
    # Conversion des tenseurs en numpy pour l'affichage
    original_np = original.cpu().numpy().squeeze()     # Format 28×28
    dropped_np = dropped.cpu().numpy().squeeze()       # Format 28×28
    reconstructed_np = reconstructed.cpu().numpy().squeeze()  # Format 28×28

    # Enregistrement de l'image originale
    plt.figure()
    plt.imshow(original_np, cmap='gray', vmin=-1, vmax=1)
    plt.title("Original")
    plt.axis('off')
    plt.savefig(f"{prefix}_original.png")
    plt.close()

    # Enregistrement de l'image modifiée (Dropped)
    plt.figure()
    plt.imshow(dropped_np, cmap='gray', vmin=-1, vmax=1)
    plt.title("Dropped")
    plt.axis('off')
    plt.savefig(f"{prefix}_dropped.png")
    plt.close()

    # Enregistrement de l'image reconstruite
    plt.figure()
    plt.imshow(reconstructed_np, cmap='gray', vmin=-1, vmax=1)
    plt.title("Reconstructed")
    plt.axis('off')
    plt.savefig(f"{prefix}_reconstructed.png")
    plt.close()

    print("Images saved as:")
    print(f"{prefix}_original.png, {prefix}_dropped.png, {prefix}_reconstructed.png")

# Récupération d'un batch depuis le test_loader
test_iter = iter(test_loader)
test_images, _ = next(test_iter)


/tmp/ipykernel_2467032/2266772600.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("mnist_autoencoder.pth", map_location=device))

In [9]:
# Sélection de l'image (1,1,28,28)
single_image = test_images[0].unsqueeze(0).to(device)

# Appel de la fonction de test :
dropped_img, reconstructed_img = test_reconstruction(
    loaded_model, 
    single_image, 
    drop_probability=0.7
)


save_reconstruction(single_image, dropped_img, reconstructed_img, "example_output.png")

Images saved as:
example_output.png_original.png, example_output.png_dropped.png, example_output.png_reconstructed.png
